<a href="https://colab.research.google.com/github/JakeOh/202105_itw_bd26/blob/main/lab_ml/ml03_train_test_set_scaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 훈련 셋과 테스트 셋을 나누는 방법
    * 순차적 추출(sequential sampling)
    * 임의 추출(random sampling)
    * 층화 추출(stratified sampling)
* 분류 모델 평가 지표
    * 정확도(accuracy)
    * 정밀도(precision)
    * 재현율(recall)
    * F1-score
* 특성 스케일링(feature scaling)
    * 표준화(standardization)
    * 정규화(normalization)

# 필요한 모듈 import

In [1]:
import numpy as np  # ndarray 함수, 메서드
import pandas as pd  # Series, DataFrame 함수, 메서드
import matplotlib.pyplot as plt  # 시각화
import seaborn as sns  # 시각화

# Machine Learning
from sklearn.neighbors import KNeighborsClassifier  # KNN 분류기(모델, 알고리즘)
from sklearn.model_selection import train_test_split  # 훈련/테스트 셋 분리 함수
from sklearn.metrics import confusion_matrix, classification_report  # 모델 평가 지표
from sklearn.preprocessing import StandardScaler, MinMaxScaler  # 특성 스케일링

# 데이터 준비

* fish.csv 파일: 어종, 생선 특성(무게, 길이, ...)
    * 문제를 간단히 하기 위해서, 어종 2개(Bream, Smelt)와 특성 2개(Weight, Length) 사용

In [2]:
fish_csv = 'https://github.com/rickiepark/hg-mldl/raw/master/fish.csv'

In [3]:
fish = pd.read_csv(fish_csv)  # CSV 파일 ---> DataFrame 생성

In [4]:
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [5]:
fish.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Species   159 non-null    object 
 1   Weight    159 non-null    float64
 2   Length    159 non-null    float64
 3   Diagonal  159 non-null    float64
 4   Height    159 non-null    float64
 5   Width     159 non-null    float64
dtypes: float64(5), object(1)
memory usage: 7.6+ KB


In [7]:
# 타겟 벡터(1차원 배열)
# 어종인 Bream 또는 Smelt인 Species 컬럼을 선택해서 numpy 배열로 변환(to_numpy() 메서드도 동일)
y = fish.loc[fish.Species.isin(['Bream', 'Smelt']), 'Species'].values
y

array(['Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt',
       'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt'],
      dtype=object)

In [8]:
y.shape  #> 원소가 49개인 1차원 배열

(49,)

In [10]:
# 타겟의 레이블 개수
np.unique(y, return_counts=True)

(array(['Bream', 'Smelt'], dtype=object), array([35, 14]))

In [13]:
# 특성 행렬(feature matrix)
# 어종이 Bream 또는 Smelt인 Weight 컬럼과 Length 컬럼을 선택하고 numpy 배열로 변환
X = fish.loc[fish.Species.isin(['Bream', 'Smelt']), ['Weight', 'Length']].values
X[:5]

array([[242. ,  25.4],
       [290. ,  26.3],
       [340. ,  26.5],
       [363. ,  29. ],
       [430. ,  29. ]])

In [14]:
X.shape  #> (49, 2) shape의 2차원 배열

(49, 2)